In [6]:
import ee
import requests
import os
# Initialize the Earth Engine API
ee.Initialize(project='ts-rishimnb')


In [7]:
import rasterio
import numpy as np
def get_data(start_date, end_date):
    """
    Downloads the NDVI band from the MODIS image collection as a GeoTIFF file for the given date range.

    Parameters:
    start_date (str): The start date in the format 'YYYY-MM-DD'.
    end_date (str): The end date in the format 'YYYY-MM-DD'.
    """
    # Define the area of interest (AOI) using coordinates (hardcoded)
    bbox = ee.Geometry.Rectangle([78.00405826, 16.93264351, 79.04971836, 17.90150706])
    
    # Choose the MODIS image collection and filter by date and bounds
    image_collection = ee.ImageCollection("MODIS/061/MOD11A1") \
        .filterBounds(bbox) \
        .filterDate(start_date, end_date)
    
    # Calculate the mean image over the date range
    img = image_collection.mean()
    
    # Generate the download URL for the NDVI band
    url = img.getDownloadUrl({
        'bands': ['LST_Night_1km'],
        'region': bbox,
        'scale': 1000,
        'format': 'GEO_TIFF'
    })
    
    # Download the file
    response = requests.get(url)
    with open('nlst3.tif', 'wb') as fd:
        fd.write(response.content)
    
    geo_tiff_path = 'nlst3.tif'
    with rasterio.open(geo_tiff_path) as src:
    # Read the first band (MODIS data can have multiple bands)
        band1 = src.read(1)
    return band1

In [8]:
print(get_data('2023-06-01', '2023-06-30'))

[[14998.42857143 15002.57142857 14995.14285714 ... 14950.55555556
  14955.33333333 14958.22222222]
 [14997.71428571 14997.42857143 14993.28571429 ... 14944.77777778
  14951.55555556 14948.        ]
 [14999.71428571 15003.14285714 15003.         ... 14940.66666667
  14948.22222222 14942.11111111]
 ...
 [14921.83333333 14931.16666667 14943.83333333 ... 14930.14285714
  14924.5        14928.2       ]
 [14913.83333333 14914.83333333 14909.83333333 ... 14909.28571429
  14910.16666667 14911.33333333]
 [14898.         14885.         14884.83333333 ... 14911.85714286
  14906.85714286 14905.33333333]]


In [9]:
# months = {
#     1: ("January", 31),
#     2: ("February", 28),
#     3: ("March", 31),
#     4: ("April", 30),
#     5: ("May", 31),
#     6: ("June", 30),
#     7: ("July", 31),
#     8: ("August", 31),
#     9: ("September", 30),
#     10: ("October", 31),
#     11: ("November", 30),
#     12: ("December", 31)
# }

# for year in range(2018,2024):
#     for i in range(1,13):
#         output_dir=fr"C:\Users\rishi\ml_projects\climate\{year}_data\nlst"
#         month_name, end_day = months[i]
#         start=f"{year}-{i}-1"
#         end = f"{year}-{i}-{end_day}"
#         arr=get_data(start,end)
#         path=os.path.join(output_dir, f'{month_name}_NLST.csv')
#         np.savetxt(path, arr, delimiter=',')

In [10]:
# import pandas as pd

# df= pd.read_csv(r"C:\Users\rishi\ml_projects\climate\2018_data\nlst\January_NLST.csv")
# df